# Notion of Maximality and Closedness

## Preamble

### Modules

`Student: Misati`

`Editor: Samuel`

In [ ]:
# [Student: Misati] Importing data mining libraries
# [Editor: Samuel] Import necessary libraries
import pandas as pd
import itertools
import random
from mlxtend.frequent_patterns import apriori, association_rules

### Configuration

`Student: Misati`

`Editor: Samuel`

In [ ]:
# [Student: Misati] Setting global configuration variables
# [Editor: Samuel] Set number of transactions to simulate
MIN_SUP = 0.3
MIN_CONF = 0.7
SEED = 123
NUM_TRANSACTIONS = 3000

In [ ]:
# [Editor: Samuel] Set a seed
random.seed(SEED)

### Utility Functions

`Student: Samuel`

`Editor: Misati`

In [ ]:
# [Student: Samuel] Generate list of transactions
# [Editor: Misati] Conversion to function

def create_transactions(item_pool):
    for _ in range(NUM_TRANSACTIONS):
    # Randomly choose 2 to 7 items per transaction
    num_items = random.randint(2, 7)
    transaction = random.sample(item_pool, num_items)
    transactions.append(transaction)

## Simulate Transaction Data

`Student: Samuel`

`Editor: Misati`

In [1]:
# [Student: Samuel] Define a pool of 30 unique supermarket items
item_pool = [
    'milk', 'bread', 'eggs', 'butter', 'cheese', 'apples', 'bananas', 'chicken', 'beef', 'rice',
    'pasta', 'sugar', 'salt', 'pepper', 'onions', 'tomatoes', 'lettuce', 'carrots', 'potatoes', 'cereal',
    'yogurt', 'juice', 'soda', 'coffee', 'tea', 'biscuits', 'chocolate', 'detergent', 'soap', 'toothpaste'
]

,Transaction
0,"[beef, eggs]"
1,"[beef, butter, chocolate, soap, detergent]"
2,"[salt, carrots]"
3,"[pasta, toothpaste, detergent, bread, apples, ..."
4,"[carrots, pasta, soda, chicken]"


In [ ]:
# [Student: Samuel] Generate a list of transactions
# [Editor: Misati] Using a utility function to generate transactions
transactions = create_transactions(item_pool)

In [ ]:
# [Student: Samuel] Convert list of transactions into a DataFrame for saving
df_transactions = pd.DataFrame({'Transaction': transactions})

In [ ]:
# [Student: Samuel] Save simulated transactions to CSV
df_transactions.to_csv('supermarket_transactions.csv', index=False)

# [Student: Samuel] preview
df_transactions.head()

## Preprocessing: One-Hot Encoding

`Student: Ambachow`

In [6]:
# [Student: Ambachow] Load the dataset
df_transactions = pd.read_csv('supermarket_transactions.csv')

In [7]:
# [Student: Ambachow] Create a set of all unique items across all transactions
unique_items = list(set(item for transaction in df_transactions['Transaction'] for item in eval(transaction)))

# [Student: Ambachow] Create a DataFrame for one-hot encoding
df_one_hot = pd.DataFrame(0, index=range(len(df_transactions)), columns=unique_items)

# [Student: Ambachow] Populate the one-hot DataFrame
for idx, transaction in enumerate(df_transactions['Transaction']):
    items = eval(transaction)  # Convert string representation of list to actual list
    for item in items:
        df_one_hot.loc[idx, item] = 1

,juice,bananas,apples,detergent,onions,tomatoes,yogurt,eggs,potatoes,biscuits,...,cheese,soda,bread,rice,beef,pasta,cereal,lettuce,tea,sugar
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [ ]:
# [Student: Ambachow] Preview the one-hot encoded DataFrame
df_one_hot.head()

### Save One-Hot Encoded Dataframe

In [ ]:
df_one_hot.to_csv('supermarket_one_hot_encoded.csv', index=False)

## Generate Frequent Itemsets

`Student: Paul`

## Identify Closed Frequent Itemsets

`Student: Misati`

## Identify Maximal Frequent Itemsets

`Student: Chawanda`

## Submission Files

`Student: Gathogo`